# Projet 1, TP3 - Optimisation avec PyTorch


En partie 1, on définit les données et une classe `torch.nn.Module` qui correspond à notre modèle.
En partie 2, on met en place une boucle d'apprentissage pour effectuer la descente de gradient sur ses paramètres, et on vérifie la cohérence du code.
En partie 3, on met en place la boucle d'apprentissage qui utilise la classe `torch.optim.Optimizer`. 
C'est la structure de code standard en PyTorch pour l'apprentissage dont on se servira par la suite.

In [ ]:
# Quelques imports utiles
import numpy as np
from numpy.polynomial.polynomial import Polynomial

import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

DEFAULT_SEED = sum(ord(c) ** 2 for c in "R5.A.12-ModMath")
torch.manual_seed(DEFAULT_SEED)

## Partie 1 - Un modèle en syntaxe PyTorch

### 1.1. Définition et génération du dataset

Les entrées $(x^{(k)})_k$ sont dans un tenseur de taille $N \times 1$, et les sorties $(y^{(k)})_k$ dans un tenseur de taille $N \times 1$. 

In [3]:
def get_dataset(
    params=(5.0, -2.0),
    x_span=(-3.0, 3.0),
    n_data=100,
    noise_amplitude=6.5,
    rng_seed=DEFAULT_SEED,
):
    slope, bias = params
    rng = torch.Generator().manual_seed(rng_seed)
    x = torch.empty(n_data, 1).uniform_(*x_span, generator=rng)
    noise = torch.empty(n_data, 1).normal_(0.0, noise_amplitude, generator=rng)
    y = slope * x + bias + noise
    return x, y


x_data, y_data = get_dataset()

### 1.2. Définir le modèle

On définit un modèle de régression linéaire à deux paramètres, $\theta = (w, b)$ avec 
$$ f_\theta(x) = w x + b , \qquad w \in \mathbb{R},\ b \in \mathbb{R} . $$
L'implémentation utilise la structure `torch.nn.Module` qui permet d'enregistrer des paramètres.
Ici on choisit des paramètres initiaux $w = 0$ et $b = 5$, qu'on cherchera à optimiser pour que le modèle colle aux données générées au sens des moindres carrés.
Cela change du TP1.1 où l'on calculait à la main la sortie du modèle.

On voit que les paramètres sont enregistrés avec la structure `torch.nn.Parameter`. On se servira de cela pour prendre de la hauteur en partie 3.

**TODO:**  
Compléter la méthode `forward` ci-dessous, qui correspond à la fonction $x \mapsto f_\theta(x)$.

In [4]:
class LinearRegression(nn.Module):
    def __init__(self, w=0.0, b=5.0):
        super().__init__()
        self.slope = nn.Parameter(torch.tensor(w))
        self.bias = nn.Parameter(torch.tensor(b))

    def forward(self, x):
        pass #TODO

### 1.3. Calcul de l'erreur moyenne

Dans le TP 1.1, on a défini la fonction `mean_squared_error`. 
Ici, on va plutôt utiliser la classe `torch.nn.MSELoss` de PyTorch, qui permet d'obtenir une fonction qui calcule la distance moyenne entre la prédiction du modèle $y_{\rm pred}^{(k)} = f_\theta(x_{\rm data}^{(k)})$ et la sortie connue $y_{\rm data}^{(k)}$,
$$ {\rm MSE}(y_{\rm pred}, y_{\rm data}) = \frac{1}{N} \sum_{k=1}^N \bigl( y_{\rm pred}^{(k)} - y_{\rm data}^{(k)} \bigr)^2 . $$

**TODO:**  
Compléter la boucle ci-dessous qui calcule la matrice `loss` pour l'affichage.

In [10]:
# valeurs de pente et de biais
w = np.linspace(-10, 10, 40)
b = np.linspace(-10, 10, 30)

loss_fun = nn.MSELoss()  # instanciation de la MSE

loss = np.zeros((len(b), len(w)))
for i in range(len(b)):
    for j in range(len(w)):
        #TODO construire un modèle avec une pente w[j] et un biais b[j]
        #TODO calculer la prédiction effectuée par ce modèle
        #TODO remplir la matrice `loss` avec l'erreur
        pass

### 1.4. Initialisation de graphes pour affichages futurs

In [13]:
x_plot, y_plot = x_data.numpy()[:, 0], y_data.numpy()[:, 0]
trace_data = go.Scatter(x=x_plot, y=y_plot, mode="markers")

contour_params = dict(
    contours_coloring="lines",
    colorscale="Greys_r",
    showscale=False,
    contours=dict(showlabels=True, labelfont=dict(size=10)),
    ncontours=40,
    line=dict(smoothing=1.3)
)
trace_loss = go.Contour(x=w, y=b, z=loss, **contour_params)

## Partie 2 - Calcul automatique des gradients, itérations manuelles

### 2.1. Itérations de descente de gradient

Pour cette partie, le code prend la forme suivante :

1. Construire le modèle de `LinearRegression` avec les paramètres par défaut.
2. Instantier la fonction de *loss* des moindres carrés.
3. Définir le learning rate $\gamma = 0.1$.
4. Effectuer des itérations de la **boucle d'apprentissage** standard :
    1. Calculer les prédictions du modèle et les comparer aux données grâce à la fonction de loss.
    2. Calculer les gradients de la *loss* par rapport à chaque paramètre avec `torch.autograd.grad`.
    3. Mettre à jour les paramètres grâce aux gradients calculés.


**TODO:**  
Remplir la cellule suivante pour suivre ces étapes. 
Penser à sauvegarder les valeurs des paramètres à chaque itération pour les afficher ensuite.
Pour copier un tenseur sans conserver son graphe de calcul, il faut l'en «détacher» avec `.detach()`.

### 2.2. Évolution des paramètres

**TODO:**  
Compléter le code d'affichage suivant.

In [ ]:
# création de la figure
fig = plotly.subplots.make_subplots(
    rows=1,
    cols=2,
    column_widths=[0.41, 0.59],
    column_titles=[
        "Erreur en fonction des paramètres",
        "Données et régressions linéaires",
    ],
)
fig.update_layout(
    width=800, height=350, margin=dict(l=20, r=20, b=20, t=20), showlegend=False
)

# topographie de l'erreur dans l'espace des paramètres
fig.add_trace(trace_loss, row=1, col=1)
fig.update_xaxes(title="w", row=1, col=1)
fig.update_yaxes(title="b", row=1, col=1)

fig.add_trace(trace_data, row=1, col=2)
fig.update_xaxes(title="x", row=1, col=2)
fig.update_yaxes(title="y", row=1, col=2)

## affichage des points sur la surface
all_colors = plotly.colors.sample_colorscale(
    plotly.colors.sequential.Plasma, samplepoints=np.linspace(0.0, 1.0, len(all_w))
)
marker_params = dict(size=10, color=all_colors)
scatter_params = dict(marker=marker_params, mode="markers", row=1, col=1)
#TODO afficher les valeurs de pente et de biais dans le plan au fil des itérations

#TODO affichage des droites (i.e. du modèle) pour chaque valeur des paramètres

fig.show()

## Partie 3 - Utilisation de la structure `Optimizer`

### 3.1. La structure d'apprentissage PyTorch

On se base sur [ce tutoriel PyTorch](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html) pour obtenir la structure du code, même si notre modèle est différent.

1. Construire le modèle de `LinearRegression` avec les paramètres par défaut.
2. Instantier la fonction de *loss* des moindres carrés.
3. Instantier l'algorithme d'optimisation avec `torch.optim.SGD` et un learning rate $\gamma = 0.1$.
4. Effectuer des itérations de la **boucle d'apprentissage** standard :
    1. Réinitialiser les gradients des paramètres du modèle avec `.zero_grad()`. Par défaut, les gradients s'additionnent ; pour éviter un double comptage, nous les remettons explicitement à zéro à chaque itération.
    2. Calculer les prédictions du modèle et les comparer aux données grâce à la fonction de loss.
    3. Rétropropager la perte de prédiction avec la méthode `.backward()` -- PyTorch dépose les gradients de la perte par rapport à chaque paramètre.
    4. Appliquer l'optimisateur avec la méthode `.step()` pour ajuster les paramètres en fonction des gradients collectés lors de la rétropropagation.

**TODO:**  
Remplir la cellule suivante pour suivre ces étapes. Penser à sauvegarder les valeurs des paramètres à chaque itération pour les afficher ensuite.

### 3.2. Affichage de l'évolution des paramètres

**TODO:**  
Compléter le code d'affichage suivant.

In [ ]:
# création de la figure
fig = plotly.subplots.make_subplots(
    rows=1,
    cols=2,
    column_widths=[0.41, 0.59],
    column_titles=[
        "Erreur en fonction des paramètres",
        "Données et régressions linéaires",
    ],
)
fig.update_layout(
    width=800, height=350, margin=dict(l=20, r=20, b=20, t=20), showlegend=False
)

# topographie de l'erreur dans l'espace des paramètres
fig.add_trace(trace_loss, row=1, col=1)
fig.update_xaxes(title="w", row=1, col=1)
fig.update_yaxes(title="b", row=1, col=1)

fig.add_trace(trace_data, row=1, col=2)
fig.update_xaxes(title="x", row=1, col=2)
fig.update_yaxes(title="y", row=1, col=2)

## affichage des points sur la surface
all_colors = plotly.colors.sample_colorscale(
    plotly.colors.sequential.Plasma, samplepoints=np.linspace(0.0, 1.0, len(all_w))
)
marker_params = dict(size=10, color=all_colors)
scatter_params = dict(marker=marker_params, mode="markers", row=1, col=1)
#TODO afficher les valeurs de pente et de biais dans le plan au fil des itérations

#TODO affichage des droites (i.e. du modèle) pour chaque valeur des paramètres

fig.show()